In this tutorial we analyse how the energy in a MLP neural network (trained with PC) could be used to assess if 
test data belongs to in-distribution or out-of-distribution. We will use the energy of the network as a measure of
surprise.

1. Prepare the in-distribution (MNIST) and out-of-distribution (FashionMNIST) datasets.
2. Training phase: Train a MLP neural network with PC on MNIST.
3. Testing phase: Test the network with MNIST and FashionMNIST.
4. Plots and tables:
    1. Plot the energy distribution histograms to visualize the distribution and spread of the energy for in-distribution and out-of-distribution data, use separate histograms for each dataset and overlay them for comparison.
    2. Create box plots to show the median, quartiles, and outliers of the energies to summarize the central tendency and variability of energies.
    3. Generate a Receiver Operating Characteristic (ROC) curve based on the energies to assess the model's ability to distinguish between in-distribution and out-of-distribution data. Plot true positive rate (sensitivity) against false positive rate (1-specificity) for different threshold values of energies.
    4. Create a table summarizing key statistics (mean, median, standard deviation) of energies for MNIST and FashionMNIST. Include columns for MNIST and FashionMNIST with rows for each statistic.
    5. Create a scatter plot of energies versus prediction confidence (max softmax value) for both MNIST and FashionMNIST. Goal is to investigate the relationship/correlation between model confidence and energy value. Use different colors or markers for MNIST and FashionMNIST data points.




In [2]:
import numpy as np

# create random dataset targets of shape (num_samples,)    
num_samples = 1000
num_classes = 10
targets = np.random.randint(0, num_classes, num_samples)
num_classes = len(np.unique(targets))
noise_level = 0.0
num_noisy = int(noise_level * len(targets))
noisy_indices = np.random.choice(len(targets), num_noisy, replace=False)
# show how many samples are noisy
print(f"Number of noisy samples: {len(noisy_indices)}")

Number of noisy samples: 0


In [16]:
# load the fashion mnist data from pytorch and compute the mean and std of the data
import torch
from torchvision import datasets, transforms

# Define the transform
transform = transforms.Compose([transforms.ToTensor()])

"""
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Define the transform with normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])
"""

# Load the dataset with the transform
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)

# Create a DataLoader to iterate through the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=len(train_data))

# Extract a batch of data
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Compute mean and std of the transformed data
mean = images.mean()
std = images.std()

# Print the mean and std of the transformed data
print(f"Mean: {mean}, Std: {std}")

# Show the min and max values of the transformed data
print(f"Min: {images.min()}, Max: {images.max()}")

Mean: 0.00011498326057335362, Std: 1.0000687837600708
Min: -0.8101983666419983, Max: 2.022662878036499


In [17]:
# load the mnist data from pytorch and compute the mean and std of the data
import torch
from torchvision import datasets, transforms

# Define the transform
transform = transforms.Compose([transforms.ToTensor()])

"""
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Define the transform with normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
"""

# Load the dataset with the transform
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)

# Create a DataLoader to iterate through the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=len(train_data))

# Extract a batch of data
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Compute mean and std of the transformed data
mean = images.mean()
std = images.std()

# Print the mean and std of the transformed data
print(f"Mean: {mean}, Std: {std}")

# Show the min and max values of the transformed data
print(f"Min: {images.min()}, Max: {images.max()}")

Mean: -0.00012828900071326643, Std: 1.0000253915786743
Min: -0.4242129623889923, Max: 2.821486711502075
